# Lecture 9: Autoencoders and Non-Negative Matrix Factorization

In this hands-on session, we'll work with ratings data for movies.  We'll use a popular dataset called ```MovieLens```, which is provided by GroupLens Research (http://grouplens.org/datasets/movielens/).  

Conceptually, the data has as a simple structure: one row per movie and one column per viewer.  For a given combination of viewer and movie, there is either a rating, or a missing value.  Ratings are on a 0 to 5 star scale, in half-star increments.

In the first half of this exerise, we'll build a neural network autoencoder that tries to summarize the ratings information for each movie.  We will then try to interpret what the autoencoder is doing (tricky).  Finally, we will cluster the movies using the representation provided by the autoencoder and applying k-Means.

In the second half of this exerise, we'll apply non-negative matrix factorization to again obtain a lower-dimensional representation of each movie.  Non-negative matrix factorization is covered in Chapter 14, Section 6, of *Elements of Statistical Learning*.  The scikit-learn documentation is also useful (http://scikit-learn.org/stable/modules/decomposition.html#nmf).

In [4]:
%matplotlib inline

AttributeError: module 'numpy' has no attribute 'core'

In [3]:
# Import the usual packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Import packages for neural networks
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.optimizers import SGD
from keras.utils import np_utils
from keras.regularizers import l2
import theano # The backend we'll be using for keras

# Import TSNE for visualization, KMeans for clustering
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans

# Import the scikit-learn module for non-negative matrix factorization
from sklearn.decomposition import NMF

AttributeError: module 'numpy' has no attribute 'core'

## Load and Transform Data

First, we'll load the ratings data.

In [ ]:
# Load the ratings data
ratings = pd.read_csv('./ratings.csv')

In [ ]:
ratings.head()

In [ ]:
ratings.shape

Now, let's reshape the ratings data so that there is one record per movie and one column per viewer.

In [ ]:
ratings_table = ratings.pivot(index='movieId', columns='userId', values='rating')

In [ ]:
# We see that there are 9066 movies, and 671 viewers.
ratings_table.shape

In [ ]:
ratings_table.head()

The documentation says that only viewers that rated at least 20 movies were included.
However, some movies may have very few ratings.  We'll throw these out in order to
make sure that each movie has a 'good' amount of ratings to work with.  There's no ideal way
to choose how many ratings are required to make the cut.  If our final goal is clustering, then we could try to cluster multiple times, using a different cutoff, and check to see how
the cutoff impacts the qualitative quality of the clusters. 

In order to find how many ratings there are per movie, I'm first going to convert the ratings table into a 0/1 data frame where the value 0 means there is no rating, and the value 1 means there is a rating.

In [ ]:
# First, let's find the missing values (which we will use to find the non-null values)
where_na = ratings_table.isnull()

In [ ]:
where_na.head()

In [ ]:
# Second, let's create an array that has a 0 where there is a missing value, and a
# 1 where is not a missing value.  We'll then be able to take column sums in order 
# to get the distribution of the number of ratings by movie.
ratings_table_01 = (-1)*(where_na.astype(int)-1)

In [ ]:
ratings_table_01.head()

In [ ]:
# Third, let's sum the number of ratings per movie
num_ratings_dist = ratings_table_01.sum(axis=1)
num_ratings_dist.shape

Before dropping movies with a small number of ratings, let's first plot a histogram of the number of reviews per movie.

In [ ]:
plt.hist(num_ratings_dist, bins='auto') 
plt.title("Histogram of Number of Ratings by Movie")
plt.show()

We see that the number of ratings distribution is very skewed.  Many movies only a few ratings, whereas a few movies have many ratings.

In [ ]:
# How many movies have only 1 rating?
(num_ratings_dist==1).sum()

In [ ]:
# How many have at least 5 ratings?
(num_ratings_dist>=5).sum()

In [ ]:
# How many have at least 10 ratings?
(num_ratings_dist>=10).sum()

I'm going to require at least 10 ratings for now.  That still leaves us 2245 movies, which is substantial.

In [ ]:
ratings_table_use = ratings_table[num_ratings_dist>=10]

In [ ]:
ratings_table_use.shape

Next, for simplicity we'll replace the missing values with 0's under the assumption that these
values represent combinations where a user was unaware of the movie.  

Note: If we were in a prediction setting, we may not want to do this. We could instead try to predict these missing values, interpreting the prediction in terms of whether or not we think a person would like the movie if they were made aware of it.  But, for the purposes of clustering, I want to simplify things and avoid missing values.

In [ ]:
ratings_table_use[ratings_table_use.isnull()] = 0

In [ ]:
ratings_table_use.head()

## Autoencoder

Now, let's create a neural network autoencoder to describe each movie in terms of underlying viewership patterns.  This amounts to dimensionality reduction, similar to PCA.  However, neural network autoencoders can perform what amounts to non-linear dimensionality reduction, making them superior in some cases.

Our autoencoder will have a very simply structure: one input layer, one hidden layer with 50 nodes, and one output layer.  The key idea behind an autoencoder is that it tries to reproduce its input. In our setting, an input corresponds to the ratings vector for a single movie.  Therefore, the output of the autoencoder must also be a ratings vector for a single movie.  Visually, if we have 671 viewers, this means that there will be 671 nodes in the input layer, and 671 nodes in the output layer.

In [ ]:
# Tell keras we want to create a sequential (feed-forward network) model, in which one
# layer follows the next
model = Sequential()

# Create the input layer and the hidden layer of the network

# 'Dense' indicates that we want all inputs to connect to every node in the hidden layer

# 'input_shape' tells the hidden layer the dimension of the input to expect, which is determined
#    by our data (the number of reviewers in our data set = 671)

#  the value '50' tells keras we want 50 hidden nodes

# 'activation' specifies how values from the input node should be processed by hidden nodes.

# 'init' tells keras how to intialize the weights for this layer (uniform distribution here)

# 'W_regularizer' tells keras to penalize the weights used to map from the inputs to the 
#  hidden layer.  As in Lasso and Ridge, it's probably good to do this in order to 
#  avoid overfitting.
model.add(Dense(50, input_shape=(671,), activation='sigmoid', init='uniform', W_regularizer=l2(0.2)))

# Next, we'll create an output layer.
# The value '671' tells keras we want this layer to have 671 output nodes, one per reviewer

# We set 'activation' to linear to tell keras not to transform the values coming out of the
# hidden layer at all.  We do that here because our true ouptuts (ratings) range between 0 and 5, so
# so forcing them to be between 0 and 1 doesn't make sense.

# Note that we don't specify 'input_shape'.  The input to this layer is the output of the hidden
# layer we created above. keras is smart enough to figure this out, which is why we only need
# to specify how many nodes are in the output layer.
model.add(Dense(671, activation='linear', init='uniform'))

# Next, we specify the properties of our optimizer.
# We'll be using stochastic gradient descent with momentum, along with 
# the mean squared error loss function (because we have continuous ratings)

# Note: The 'decay' argument was not discussed in lecture - it reduces the learning
# rate as we get further into training (e.g., as we go from one epoch to the next)
# The benefit of decay is that the optimizer will make bigger adjustments
# early on, then do fine-tuning later in the training process.
sgd = SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='mse', optimizer=sgd)

# Summarize the model setup
model.summary()

Now let's fit the model.  Unlike in a supervised learning problem, we don't have actual response data.  The goal of the autoencoder is to reconstruct the input data.  Therefore, we tell keras that both the input and output data are the ratings data.  We will also tell keras to report loss on a validation set, so we can track out of sample
performance.  Keras lets you manually specify a validation set, or you can just tell it what percent of the data to hold out.

In [ ]:
model.fit(np.array(ratings_table_use), np.array(ratings_table_use), batch_size=100, nb_epoch=200, validation_split=0.2)

When setting up this notebook, the following parameters worked decently well: lr=0.001, decay=1e-6, momentum=0.9, nesterov=True, batch_size = 100, nb_epoch = 200, using validation split of 0.2.  Validation error bottomed out at 105 epochs.  Therefore, I'm going to refit once on all the data using 105 epochs, and use those weights moving forward.

In [ ]:
model.fit(np.array(ratings_table_use), np.array(ratings_table_use), batch_size=100, nb_epoch=105)

Now, I want to try and interpret the nodes.  To do so, I'm going to look for the movies that load most heavily on each node.  But, in order to do this, I need to calculate the activation on each node for each movie.  We'll calculate these activations manually using the weights that map from the input layer to the hidden layer, and the bias.  keras lets us access these directly using ```get_weights```. 

Regarding the weights, since the input layer has 671 nodes (one per reviewer), and the hidden layer has 50 nodes, we should expect a weights matrix with 671 rows and 50 columns representing one weight per combination of viewere and hidden node.

In [ ]:
model.get_weights()[0].shape

Regarding the biases, since there are 50 hidden nodes, we should expect 50 bias terms -one per node.

In [ ]:
model.get_weights()[1].shape

In [ ]:
# Lets store the weights and biases
ae_weights = model.get_weights()[0]
ae_bias = model.get_weights()[1]

Next, we map each movie's raw input to its hidden layer representation.  We can trick keras into doing this for us, but we can also do it manually.  Recall that, for a given movie with ratings $x_{1},\dots,x_{V}$ (where $V$ = the number of viewers), a hidden node using sigmoid activation with weights $w_{1},\dots,w_{V}$ and bias $b$ performs the following computation
$$ \frac{1}{1+e^{-\left(b+\sum_{v=1}^{V}w_{v}x_{v}\right)}}. $$

We'll just perform this computation manually for all movies at once using matrix multiplication.

In [ ]:
weight_prod = np.dot(np.array(ratings_table_use),np.array(ae_weights)) # Multiply inputs by weights
add_bias = np.tile(np.array(ae_bias),[2245,1]) # Add bias

h_layer = 1/(1+np.exp(weight_prod+add_bias)) # Finally, transform via sigmoid activation

In [ ]:
# Convert resulting numpy array to df so we don't forget which movies correpond to each row
h_layer = pd.DataFrame(data=h_layer,index=ratings_table_use.index) 

In [ ]:
h_layer.head()

Now, let's try to interpret what the hidden layer is doing.  First, we'll produce a heatmap of the movie x node table shown above.  We'll filter to only the first 50 movies since there are over 2k movies.

In [ ]:
heatmap = plt.pcolor(h_layer.ix[1:50,:])

Already, we can see that there are some movies that load heavily on most, if not all, nodes (corresponding to dark rows in the heatmap).

Next, we'll find the movies that score the highest on each of the nodes.  To do this, we'll need to map the movieId's to names, so let's import the ```movies.csv``` file.

In [ ]:
movies = pd.read_csv('./movies.csv')
movies.head()

In [ ]:
# Let's drop the defalt index and instead use movieID, which will make it easier to lookup values
# in this table based on movieID's in the hidden layer table.
movies = movies.set_index(['movieId'])

In [ ]:
movies.head()

Let's write a function that takes in a hidden node number and prints out the top scoring movies for that node.

In [ ]:
def interp_node(node_num, top_n):
    
    # First, get the top_n movie ID's for this node
    top_n_movies = h_layer.ix[:,node_num].sort_values(ascending=False)[:top_n].index.values
    
    # Second, find the movie names corresponding to those movieIDs
    print(movies.ix[top_n_movies,0])

Now, let's loop over the nodes, printing out top movies for each ones.

In [ ]:
for node_num in range(50):
    print("Node " + str(node_num))
    interp_node(node_num,10)
    print("\n")

It seems like all nodes are primarily picking up the most popular movies.  That's not encouraging, but it doesn't mean that clustering on the hidden node representation will do badly, since clustering will account for a movie's score's across all hidden nodes, not just one.

Next, let's try to understand which reviewers are defining the transformation represented by each node.  Recall that a movie's score for a given hidden node is determined by a linear combination of that movie's reviews across reviewers.  Therefore, it's possible that each hidden node pays attention to a different set of viewers, or at least applies different weights to the viewers.

In order to analyze how each node weights each viewer, we need to go back to the weights matrix we extracted before.  We called it ``ae_weights``, and recall that it has one row per viewer, and one column per hidden node.  Therefore, for a given hidden node, we just need to find the viewers with the highest weights.  The code required is very similar to what we used to identify top scoring movies for each node.

In [ ]:
# Adding an index to the weights so I can look up by viewer id.
ae_weights_df = pd.DataFrame(data=ae_weights,index=range(1,672))

In [ ]:
ae_weights_df.head()

In [ ]:
def interp_node_viewer(node_num, top_n):
    
    # First, get the top_n movie ID's for this node
    top_n_viewers = ae_weights_df.ix[:,node_num].sort_values(ascending=False)[:top_n].index.values
    return top_n_viewers

In [ ]:
for node_num in range(50):
    print("Node " + str(node_num))
    print(interp_node_viewer(node_num,10))
    print("\n")

It looks like a handful of viewers are influential in that they have the highest weights for most nodes.  Let's check by conunting, for each viewer, the number of times they appear in the top 10 viewers across all nodes.

In [ ]:
# Let's create a dictionary with one entry per viewer that will track
# the number of times a viewer appears in the top ten viewers for a hidden node.
viewer_count = dict((x,0) for x in range(672))

In [ ]:
for node_num in range(50):
    top_viewers = interp_node_viewer(node_num,10)
    for viewer in range(672):
        if viewer in top_viewers:
            viewer_count[viewer] += 1

In [ ]:
viewer_count

We see that a small handful of viewers tend to have the biggest impact on most nodes. Again, not entirely encouraging, but we don't want to look at one viewer in isolation.

## k-Means Clustering Using the Autoencoder Representation

Now, we'll cluster the movies using k-means. Note that each movie is now represented as a vector in 50 dimensional space (one coordinate per hidden node).  

In [ ]:
num_clusters = 20
k_mns = KMeans(n_clusters=num_clusters, n_init=10).fit(np.array(h_layer))

In [ ]:
k_mns.labels_

Next, we'll run the hidden node representation of the movies through t-SNE, in order to embed the movies in two-dimensional space.  This will let us plot the movies, and ensures that the 2-d representation we get is representative of the actual 50-dimensional respresentation that was used for clustering.

In [ ]:
TSNE_embed = TSNE(n_components=2, random_state=0)
TSNE_embed = TSNE_embed.fit_transform(np.array(h_layer)) 

In [ ]:
TSNE_embed.shape

Next, we'll plot the movies in 2-d space (using the t-SNE coordinates), and color the movies by cluster.  Unfortunately, there aren't many colors to choose from, so some clusters will get the same color.

In [ ]:
from matplotlib.pyplot import cm 
color=cm.rainbow(np.linspace(0,1,num_clusters))

fig = plt.figure()
fig.set_figheight(15)
fig.set_figwidth(15)
ax = fig.add_subplot(111)

for clust in range(num_clusters):
    ax.scatter(TSNE_embed[k_mns.labels_==clust,0],TSNE_embed[k_mns.labels_==clust,1], c=color[clust], label='Cluster '+str(clust))
    
plt.legend(loc='upper right')
plt.show()


Let's see if we can get any clarity by plotting in three dimensions.  To do this, we'll use t-SNE to map the 50-dimensional representation into three dimensions instead of 2, then plot that 3-d representation with colors indicating the k-means cluster.

In [ ]:
# Re-run t-SNE projecting into 3 dimensions
TSNE_embed_3 = TSNE(n_components=3, random_state=0)
TSNE_embed_3 = TSNE_embed_3.fit_transform(np.array(h_layer)) 

In [ ]:
# Plot the movies in 3-d space
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure()
fig.set_figheight(15)
fig.set_figwidth(15)
ax = fig.add_subplot(111, projection='3d')
for clust in range(num_clusters):
    ax.scatter(TSNE_embed_3[k_mns.labels_==clust,0],TSNE_embed_3[k_mns.labels_==clust,1], TSNE_embed_3[k_mns.labels_==clust,2], c=color[clust], label='Cluster '+str(clust))
plt.legend(loc='upper left')
plt.show()

Not a huge amount of clarity unfortunately.  If this were a real project, we'd need to further tune our model, or try different approaches.  We'll try one such approach, non-negative matrix factorization, below.  However, any ideas on what we could do differently to improve the analysis?

## Non-Negative Matrix Factorization

In this section, we'll apply non-negative matrix factorization.  The idea is to find a set of *latent* features that represent a film in terms of its ratings.  

First, we insantiate an NMF object and specify the parameters we want to use.  *Number of components* corresponds to the number of latent features we want to use, analogous to the numbder of hidden nodes in the auto encoder. *alpha* and *l1_ratio* are regularization parameters used to avoid overfitting.  *max_iter* is the maximum number of iterations NMF will attempt when trying to perform the factorization.  

In [ ]:
model_nmf = NMF(n_components=50, alpha=0.1, l1_ratio = 0.1, max_iter = 500)

Second, we perform the decomposition and transform the movie data at the same time. The result will be a matrix with one record per movie, and one column per latent feature.

In [ ]:
movies_decomp = model_nmf.fit_transform(ratings_table_use.as_matrix())

In [ ]:
movies_decomp.shape

In [ ]:
movies_decomp

Now, let's try to understand the latent features. We'll start by producing a heatmap that shows how movies load on the 50 latent features.

In [ ]:
heatmap = plt.pcolor(movies_decomp[1:50,:])

Lots more variation than we got with the autoencoder.  Next, let's try to find the movies that load most heavily on each of the latent featres.  We'll use a very similar function to what we used with the autoencoder.

In [ ]:
movies_decomp_df = pd.DataFrame(data=movies_decomp,index=ratings_table_use.index) 

In [ ]:
def interp_feature_nmf(feature_num, top_n):
    
    # First, get the top_n movie ID's for this node
    top_n_movies = movies_decomp_df.ix[:,feature_num].sort_values(ascending=False)[:top_n].index.values
    
    # Second, find the movie names corresponding to those movieIDs
    print(movies.ix[top_n_movies,0])

In [ ]:
for feature_num in range(50):
    print("Feature " + str(feature_num))
    interp_feature_nmf(feature_num,10)
    print("\n")

It looks like NMF is doing a better job than the autoencoder.  The most popular movies aren't popping up over and over, and there generally seems to be genre associated with some of the features.

We'll now repeat the t-SNE + k-Means analysis using the 50-dimensional latent representation for the movies.

In [ ]:
k_mns_NMF = KMeans(n_clusters=num_clusters, n_init=10).fit(movies_decomp)

In [ ]:
k_mns_NMF.labels_

In [ ]:
TSNE_embed_NMF = TSNE(n_components=2, random_state=0)
TSNE_embed_NMF = TSNE_embed_NMF.fit_transform(movies_decomp)

In [ ]:
TSNE_embed_NMF.shape

In [ ]:
color=cm.rainbow(np.linspace(0,1,num_clusters))

fig = plt.figure()
fig.set_figheight(15)
fig.set_figwidth(15)
ax = fig.add_subplot(111)

for clust in range(num_clusters):
    ax.scatter(TSNE_embed_NMF[k_mns_NMF.labels_==clust,0],TSNE_embed_NMF[k_mns_NMF.labels_==clust,1], c=color[clust], label='Cluster '+str(clust))
    
plt.legend(loc='upper right')
plt.show()

In [ ]:
# Re-run t-SNE projecting into 3 dimensions
TSNE_embed_3_NMF = TSNE(n_components=3, random_state=0)
TSNE_embed_3_NMF = TSNE_embed_3_NMF.fit_transform(movies_decomp) 

In [ ]:
# Plot the movies in 3-d space
fig = plt.figure()
fig.set_figheight(15)
fig.set_figwidth(15)
ax = fig.add_subplot(111, projection='3d')
for clust in range(num_clusters):
    ax.scatter(TSNE_embed_3_NMF[k_mns_NMF.labels_==clust,0],TSNE_embed_3_NMF[k_mns_NMF.labels_==clust,1], TSNE_embed_3_NMF[k_mns_NMF.labels_==clust,2], c=color[clust], label='Cluster '+str(clust))
plt.legend(loc='upper left')
plt.show()

In conclusion, it seems like the NMF's latent features make more sense, but don't do as good of a job at separating the movies.  The autoencoder, at least from what we can tell from the t-sne representation, more clearly separates the movies, but the hidden nodes are harder to interpret.  In either case, we should continue refining our analysis.